In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660605 sha256=4f567032692e9d4fd1a595a251b2c8383cb1364dc9fa3265d6d63062d071b0da
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [2]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import struct

mod = SourceModule("""
    #include <math.h>
    #define MAX_DEPTH 3
    #define PI 3.14159265358979323846

    struct Vec3 {
        float x, y, z;
        __device__ Vec3 operator+(const Vec3& v) const {
            return {x + v.x, y + v.y, z + v.z};
        }
        __device__ Vec3 operator-(const Vec3& v) const {
            return {x - v.x, y - v.y, z - v.z};
        }
        __device__ Vec3 operator*(float scalar) const {
            return {x * scalar, y * scalar, z * scalar};
        }
        __device__ Vec3 operator*(const Vec3& v) const {
            return {x * v.x, y * v.y, z * v.z};
        }
        __device__ float dot(const Vec3& v) const {
            return x * v.x + y * v.y + z * v.z;
        }
        __device__ Vec3 normalize() const {
            float len = sqrtf(dot(*this));
            return (len > 0) ? *this * (1.0f / len) : Vec3{0.0f, 0.0f, 0.0f};
        }
        __device__ Vec3& operator+=(const Vec3& v) {
            x += v.x;
            y += v.y;
            z += v.z;
            return *this;
        }
    };
    struct Sphere {
        Vec3 center;
        float radius;
        Vec3 color;
    };
    struct Light {
        Vec3 position;
        Vec3 intensity;
    };
    struct Plane {
        Vec3 point;
        Vec3 normal;
        Vec3 color;
    };
    __device__ bool intersect(const Vec3& rayOrigin, const Vec3& rayDir, const Sphere& sphere, float& t) {
        Vec3 oc = rayOrigin - sphere.center;
        float a = rayDir.dot(rayDir);
        float b = 2.0f * oc.dot(rayDir);
        float c = oc.dot(oc) - sphere.radius * sphere.radius;
        float discriminant = b * b - 4 * a * c;
        if (discriminant < 0) return false;
        t = (-b - sqrtf(discriminant)) / (2.0f * a);
        return t >= 0;
    }

    __device__ bool intersectPlane(const Vec3& rayOrigin, const Vec3& rayDir, const Plane& plane, float& t) {
        float denom = plane.normal.dot(rayDir);
        if (fabs(denom) > 1e-6f) {
            Vec3 p0l0 = plane.point - rayOrigin;
            t = p0l0.dot(plane.normal) / denom;
            return (t >= 0);
        }
        return false;
    }

    __device__ Vec3 traceRay(const Vec3& rayOrigin, const Vec3& rayDir, Sphere* spheres, int numSpheres, Light* lights,
    int numLights, Plane* planes, int numPlanes, int depth) {
        if (depth > MAX_DEPTH) return {0.0f, 0.0f, 0.0f};
        float closestT = 1e20f;
        int closestSphere = -1;
        int closestPlane = -1;
        bool hitPlane = false;
        for (int i = 0; i < numSpheres; ++i) {
            float t;
            if (intersect(rayOrigin, rayDir, spheres[i], t) && t < closestT) {
                closestT = t;
                closestSphere = i;
            }
        }

        for (int i = 0; i < numPlanes; ++i) {
            float t;
            if (intersectPlane(rayOrigin, rayDir, planes[i], t) && t < closestT) {
                closestT = t;
                closestPlane = i;
                hitPlane = true;
            }
        }

        Vec3 color = {0.0f, 0.0f, 0.0f};
        if (hitPlane && closestPlane != -1) {
            const Plane& plane = planes[closestPlane];
            Vec3 intersection = rayOrigin + rayDir * closestT;
            Vec3 normal = plane.normal.normalize();
            color = plane.color * 0.1f;
            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();
                float brightness = fmaxf(0.0f, normal.dot(lightDir));
                color += plane.color * (lights[i].intensity * brightness);
            }
        }

        if (closestSphere != -1) {
            const Sphere& sphere = spheres[closestSphere];
            Vec3 intersection = rayOrigin + rayDir * closestT;
            Vec3 normal = (intersection - sphere.center).normalize();
            color = sphere.color * 0.1f;
            for (int i = 0; i < numLights; ++i) {
                Vec3 lightDir = (lights[i].position - intersection).normalize();
                bool inShadow = false;
                for (int j = 0; j < numSpheres; ++j) {
                    float shadowT;
                    if (intersect(intersection + normal * 1e-4f, lightDir, spheres[j], shadowT)) {
                        inShadow = true;
                        break;
                    }
                }
                if (!inShadow) {
                    float brightness = fmaxf(0.0f, normal.dot(lightDir));
                    color += sphere.color * (lights[i].intensity * brightness);
                }
            }
        }
        return color;
    }

__global__ void renderKernel(unsigned char* image, int width, int height, Sphere* spheres, int numSpheres,
Light* lights, int numLights, Plane* planes, int numPlanes) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    // Проверка выхода за границы изображения
    if (x >= width || y >= height) return;

    // Вычисление координат луча
    float aspectRatio = (float)width / height;
    float rayDirX = (2.0f * (x + 0.5f) / width - 1.0f) * aspectRatio;
    float rayDirY = 1.0f - 2.0f * (y + 0.5f) / height;
    Vec3 rayOrigin = {0.0f, 0.0f, 0.0f};

    // Создаем вектор направления и нормализуем его
    Vec3 rayDir = {rayDirX, rayDirY, -1.0f};
    rayDir.normalize(); // Нормализация вектора
    // Трассировка луча
    Vec3 color = traceRay(rayOrigin, rayDir, spheres, numSpheres, lights, numLights, planes, numPlanes, 0);
    // Преобразование цвета в формат изображения
    int pixelIndex = (y * width + x) * 3;
    image[pixelIndex]     = (unsigned char)(fminf(1.0f, color.x) * 255);
    image[pixelIndex + 1] = (unsigned char)(fminf(1.0f, color.y) * 255);
    image[pixelIndex + 2] = (unsigned char)(fminf(1.0f, color.z) * 255);
}
""")


def create_scene():
    spheres = np.array([
        (np.array([0.0, -1.0, -3.0], dtype=np.float32), 1.0, np.array([1.0, 0.0, 0.0], dtype=np.float32)),
        (np.array([2.0, 0.0, -4.0], dtype=np.float32), 1.0, np.array([0.0, 1.0, 0.0], dtype=np.float32)),
        (np.array([-2.0, 0.0, -4.0], dtype=np.float32), 1.0, np.array([0.0, 0.0, 1.0], dtype=np.float32)),
        (np.array([1.0, -0.5, -5.0], dtype=np.float32), 1.0, np.array([1.0, 1.0, 0.0], dtype=np.float32)),
    ], dtype=[('center', np.float32, 3), ('radius', np.float32), ('color', np.float32, 3)])
    lights = np.array([
        (np.array([2.0, 2.0, -3.0], dtype=np.float32), np.array([1.0, 1.0, 1.0], dtype=np.float32))
    ], dtype=[('position', np.float32, 3), ('intensity', np.float32, 3)])
    planes = np.array([
        (np.array([0.0, -2.0, 0.0], dtype=np.float32), np.array([0.0, 1.0, 0.0], dtype=np.float32),
         np.array([0.5, 0.5, 0.5], dtype=np.float32))
    ], dtype=[('point', np.float32, 3), ('normal', np.float32, 3), ('color', np.float32, 3)])
    return spheres, lights, planes


def save_bmp(filename, image, width, height):
    header = bytearray()
    header.extend(b'BM')  # Signature 'BM'
    file_size = 54 + len(image)  # Размер файла (заголовок + пиксели)
    header.extend(struct.pack('<I', file_size))  # Общий размер файла
    header.extend(b'\x00\x00\x00\x00')  # Резерв
    header.extend(struct.pack('<I', 54))  # Смещение до данных пикселей
    header.extend(struct.pack('<I', 40))  # Размер DIB-заголовка
    header.extend(struct.pack('<I', width))  # Ширина
    header.extend(struct.pack('<I', height))  # Высота
    header.extend(b'\x01\x00')  # Количество цветовых плоскостей
    header.extend(b'\x18\x00')  # Количество бит на пиксель (24 бита)
    header.extend(b'\x00\x00\x00\x00')  # Без сжатия
    header.extend(b'\x00\x00\x00\x00')  # Размер изображения (для не сжимаемых данных = 0)
    header.extend(b'\x00\x00\x00\x00')  # Разрешение по оси X
    header.extend(b'\x00\x00\x00\x00')  # Разрешение по оси Y
    header.extend(b'\x00\x00\x00\x00')  # Число используемых цветов
    header.extend(b'\x00\x00\x00\x00')  # Число важных цветов

    with open(filename, 'wb') as f:
        f.write(header)
        for row in range(height - 1, -1, -1):
            for col in range(width):
                idx = (row * width + col) * 3
                b, g, r = image[idx], image[idx + 1], image[idx + 2]
                f.write(bytes([b, g, r]))


def main(width, height, output_file):
    spheres, lights, planes = create_scene()
    numSpheres = len(spheres)
    numLights = len(lights)
    numPlanes = len(planes)
    spheres_gpu = cuda.mem_alloc(spheres.nbytes)
    lights_gpu = cuda.mem_alloc(lights.nbytes)
    planes_gpu = cuda.mem_alloc(planes.nbytes)
    cuda.memcpy_htod(spheres_gpu, spheres)
    cuda.memcpy_htod(lights_gpu, lights)
    cuda.memcpy_htod(planes_gpu, planes)

    image = np.zeros((height, width, 3), dtype=np.uint8)
    image_gpu = cuda.mem_alloc(image.nbytes)
    block_size = (16, 16, 1)
    grid_size = ((width + block_size[0] - 1) // block_size[0],
                 (height + block_size[1] - 1) // block_size[1])
    render_kernel = mod.get_function("renderKernel")
    render_kernel(image_gpu, np.int32(width), np.int32(height), spheres_gpu, np.int32(numSpheres), lights_gpu,
                  np.int32(numLights), planes_gpu, np.int32(numPlanes),
                  block=block_size, grid=grid_size)
    cuda.memcpy_dtoh(image, image_gpu)
    save_bmp(output_file, image.flatten(), width, height)


if __name__ == "__main__":
    main(800, 600, "output.bmp")